### Get Data Code

### Last Updated: 12 Oct 2023

### Authors: Momona Yamagami, Maneeshika Madduri

### Goal: 
This code take the h5 files that the partcipant data was saved in and coverts it to a pickle file along with the experimental conditions. For the sake of memory, the h5 files were split into block 1 and block 2, each participant has block 1 and block 2 data. 

In [21]:
import aopy 
#aopy as in Orsborn lab's data processing pipeline, 
#can be downloaded here https://github.com/aolabNeuro/analyze
import numpy as np
import matplotlib.pyplot as plt
import seaborn 
import scipy
from scipy.optimize import minimize,least_squares
import copy as copy
import time as tpy
import glob
import os
import pandas as pd
import pickle

seaborn.set()

# Co-adaptive Myoelectric Interface Continuous Task data extraction
## extract data and save to pandas dataframe

- 14 participants (METAS106 - METAS119)
- 2 blocks (BLOCK1, BLOCK2)
- 
- `weiner_task_data` $\rightarrow$ data that we want
- `quattro_data` $\rightarrow$ pretty sure this is raw EMG data

ISSUE: can't analyze block 1 AND block 2 -- memory issue

In [2]:
keys = ['METACPHS_S106', 
        'METACPHS_S107',
        'METACPHS_S108', 
        'METACPHS_S109', 
        'METACPHS_S110', 
        'METACPHS_S111', 
        'METACPHS_S112',
        'METACPHS_S113',
        'METACPHS_S114',
        'METACPHS_S115',
        'METACPHS_S116',
        'METACPHS_S117',
        'METACPHS_S118',
        'METACPHS_S119']

# create dataframe
var_names = ['refs','poss','dec_vels','int_vels','emgs','Ws','Hs','alphas', 'pDs','times','conditions']

## BLOCK 1
refs_block1 = {key: [] for key in keys} # re
poss_block1 = {key: [] for key in keys}
dec_vels_block1 = {key: [] for key in keys}                                            
emgs_block1 = {key: [] for key in keys}
int_vel_block1 = {key: [] for key in keys}
Ws_block1 = {key: [] for key in keys}
Hs_block1 = {key: [] for key in keys}
alphas_block1 = {key: [] for key in keys}
pDs_block1 = {key: [] for key in keys}
times_block1 = {key: [] for key in keys}
conditions_block1 = {key: [] for key in keys}

## BLOCK 2
refs_block2 = {key: [] for key in keys} # re
poss_block2 = {key: [] for key in keys}
dec_vels_block2 = {key: [] for key in keys}
emgs_block2 = {key: [] for key in keys}
int_vel_block2 = {key: [] for key in keys}
Ws_block2 = {key: [] for key in keys}
Hs_block2 = {key: [] for key in keys}
alphas_block2 = {key: [] for key in keys}
pDs_block2 = {key: [] for key in keys}
times_block2 = {key: [] for key in keys}
conditions_block2 = {key: [] for key in keys}
# decoders_block1 = {key: [] for key in keys}

In [22]:
# what are the subjects? confirm the subject numbers
keys

['METACPHS_S106',
 'METACPHS_S107',
 'METACPHS_S108',
 'METACPHS_S109',
 'METACPHS_S110',
 'METACPHS_S111',
 'METACPHS_S112',
 'METACPHS_S113',
 'METACPHS_S114',
 'METACPHS_S115',
 'METACPHS_S116',
 'METACPHS_S117',
 'METACPHS_S118',
 'METACPHS_S119']

First need to sort files in order by learning rate and then decoder condtion. We expect this to be the order of files as they are saved:

a_25_D_1 

a_25_D_2

a_25_D_5

a_25_D_6

a_75_D_3

a_75_D_4

a_75_D_7

a_75_D_8

Then find the indices of the filenames required to sort the data

In [24]:
# set directory where the data is stored
PATH =  '/Volumes/My Passport/cphs/'

In [25]:
files = []
print(PATH)
data_dir = PATH + 'METACPHS_S112' +'/BLOCK2'
print(data_dir)
for name in glob.glob(data_dir + '/weiner_task_data*.h5'):
    file_name = os.path.basename(name)
    files.append(file_name)

print(files)
test_file = files[0]
a_num = 7 # index of alpha
d_num = 9 # index of decoder condition
test_file.split("_")[a_num] + test_file.split("_")[d_num]  # choose indexes 7 and 9
files = sorted(files, key = lambda x: x.split("_")[a_num] + x.split("_")[d_num])
print("sorted:")
print(files)

/Volumes/My Passport/cphs/
/Volumes/My Passport/cphs/METACPHS_S112/BLOCK2
['weiner_task_data_METACPHS_S112_L2_a_25_D_1_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_25_D_2_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_25_D_5_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_75_D_3_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_75_D_4_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_75_D_7_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_75_D_8_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_25_D_6_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5']
sorted:
['weiner_task_data_METACPHS_S112_L2_a_25_D_1_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_25_D_2_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5', 'weiner_task_data_METACPHS_S112_L2_a_25_D_5_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-

T

In [5]:
print('analyze block 1')
for key in keys: # each subject
    print('analyzing', key)
    data_dir = PATH + key +'/BLOCK1' # get you to the folder
    files = [] # set up list of files to sort
    for name in glob.glob(data_dir + '/weiner_task_data*.h5'):
        file_name = os.path.basename(name)
        files.append(file_name)
    

    # sort the files in the order we expect
    files = sorted(files, key = lambda x: x.split("_")[a_num] + x.split("_")[d_num])  

    for file_name in files:
        weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
        f = weiner_task_data
        c = weiner_config
#         print(f.dtype.names) # get the names of the fields
        refs_block1[key].append(np.asarray(f['reference']))
        poss_block1[key].append(np.asarray(f['decoded_position']))
        dec_vels_block1[key].append(np.asarray(f['decoded_velocity']))
        int_vel_block1[key].append(np.asarray(f['intended_velocity'])) # didn't seem to be saving
        emgs_block1[key].append(np.asarray(f['filtered_emg']))
        Ws_block1[key].append(np.asarray(f['weiner_filter_w']))
        Hs_block1[key].append(np.asarray(f['weiner_filter_h'][0]))
        alphas_block1[key].append(c['alpha'])
        pDs_block1[key].append(c['lambdaD'])
        time = np.asarray(f['timestamp']).flatten()
        times_block1[key].append(time - time[0])
        conditions_block1[key].append(file_name[-41:-38])
    refs_block1[key] = np.asarray(refs_block1[key])
    poss_block1[key] = np.asarray(poss_block1[key])
    dec_vels_block1[key] = np.asarray(dec_vels_block1[key])
    emgs_block1[key] = np.asarray(emgs_block1[key])
    Ws_block1[key] = np.asarray(Ws_block1[key])
    Hs_block1[key] = np.asarray(Hs_block1[key])
    alphas_block1[key] = np.asarray(alphas_block1[key])
    pDs_block1[key] = np.asarray(pDs_block1[key])
    times_block1[key] = np.asarray(times_block1[key])
    conditions_block1[key] = np.asarray(conditions_block1[key])
    int_vel_block1[key] = np.asarray(int_vel_block1[key])

    assert(np.all(conditions_block1[key] == ['D_1', 'D_2', 'D_5', 'D_6', 'D_3', 'D_4', 'D_7', 'D_8']) == True)
    assert(len(conditions_block1[key])== 8) 

analyze block 1
analyzing METACPHS_S106
analyzing METACPHS_S107
analyzing METACPHS_S108
analyzing METACPHS_S109
analyzing METACPHS_S110
analyzing METACPHS_S111
analyzing METACPHS_S112
analyzing METACPHS_S113
analyzing METACPHS_S114
analyzing METACPHS_S115
analyzing METACPHS_S116
analyzing METACPHS_S117
analyzing METACPHS_S118
analyzing METACPHS_S119


In [6]:
# make sure files are in the order we want them to be in 

print('analyze block 2')
for key in keys: # each subject
    print('analyzing', key)
    data_dir = PATH + key +'/BLOCK2' # get you to the folder
    files = [] # set up list of files
    for name in glob.glob(data_dir + '/weiner_task_data*.h5'):
        file_name = os.path.basename(name)
        files.append(file_name)

    # sort files in order we expect
    files = sorted(files, key = lambda x: x.split("_")[a_num] + x.split("_")[d_num])  

    for file_name in files:
        weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
        f = weiner_task_data
        c = weiner_config
        refs_block2[key].append(np.asarray(f['reference']))
        poss_block2[key].append(np.asarray(f['decoded_position']))
        dec_vels_block2[key].append(np.asarray(f['decoded_velocity']))
        int_vel_block2[key].append(np.asarray(f['intended_velocity'])) # didn't seem to be saving
        emgs_block2[key].append(np.asarray(f['filtered_emg']))
        Ws_block2[key].append(np.asarray(f['weiner_filter_w']))
        Hs_block2[key].append(np.asarray(f['weiner_filter_h'][0]))
        alphas_block2[key].append(c['alpha'])
        pDs_block2[key].append(c['lambdaD'])
        time = np.asarray(f['timestamp']).flatten()
        times_block2[key].append(time - time[0])
        conditions_block2[key].append(file_name[-41:-38])
    refs_block2[key] = np.asarray(refs_block2[key])
    poss_block2[key] = np.asarray(poss_block2[key])
    dec_vels_block2[key] = np.asarray(dec_vels_block2[key])
    emgs_block2[key] = np.asarray(emgs_block2[key])
    Ws_block2[key] = np.asarray(Ws_block2[key])
    Hs_block2[key] = np.asarray(Hs_block2[key])
    alphas_block2[key] = np.asarray(alphas_block2[key])
    pDs_block2[key] = np.asarray(pDs_block2[key])
    times_block2[key] = np.asarray(times_block2[key])
    conditions_block2[key] = np.asarray(conditions_block2[key])
    int_vel_block2[key] = np.asarray(int_vel_block2[key])
    assert(np.all(conditions_block2[key] == ['D_1', 'D_2', 'D_5', 'D_6', 'D_3', 'D_4', 'D_7', 'D_8']) == True)
    assert(len(conditions_block2[key])== 8) 

analyze block 2
analyzing METACPHS_S106
analyzing METACPHS_S107
analyzing METACPHS_S108
analyzing METACPHS_S109
analyzing METACPHS_S110
analyzing METACPHS_S111
analyzing METACPHS_S112
analyzing METACPHS_S113
analyzing METACPHS_S114
analyzing METACPHS_S115
analyzing METACPHS_S116
analyzing METACPHS_S117
analyzing METACPHS_S118
analyzing METACPHS_S119


In [20]:
# confirm the directory where we want to save the data
PATH

'/Volumes/My Passport/cphs/'

In [18]:
data = [refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1,Hs_block1,alphas_block1,pDs_block1,times_block1,conditions_block1]

with open(PATH + 'continuous_full_data_block1_sorted.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
data = [refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2]

with open(PATH + 'continuous_full_data_block2_sorted.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)